In [ ]:
import geoviews as gv
import panel as pn
import pystac

import utils.xyt
import utils.zarr

pn.extension()
gv.extension("bokeh")

In [ ]:
CATALOG_URL = "https://s3.waw3-2.cloudferro.com/swift/v1/wpl-stac/stac/catalog.json"
SITE = "degero"
COLLECTION = "albedo"

In [ ]:
root: pystac.Catalog = pystac.read_file(CATALOG_URL)  # type: ignore
catalog: pystac.Catalog = root.get_child(SITE)  # type: ignore
collection: pystac.Collection = catalog.get_child(COLLECTION)  # type: ignore

In [ ]:
extent = utils.xyt.Extent.from_pystac(collection.extent)

In [ ]:
xyt = utils.xyt.XYT(extent=extent)

In [ ]:
obj = utils.zarr.ZarrDataset.from_pystac(xyt, collection)

In [ ]:
obj

In [ ]:
xyt